# TUI Template - Design Only

Prototype for the TUI design. Exported to `tui_writer/tui_template.py` but NOT part of main package.

**Run with:** `python -m tui_writer.tui_template`

In [ ]:
#| default_exp tui_template

In [ ]:
#| export
from textual.app import App, ComposeResult
from textual.containers import Container
from textual.widgets import Header, Footer, Static

In [ ]:
#| export
class TUITemplate(App):
    """Standalone TUI template for design testing."""

    CSS = """
    Screen {
        align: center middle;
    }
    """

    BINDINGS = [("q", "quit", "Quit")]
    
    def compose(self) -> ComposeResult:
        yield Header()
        yield Static("TUI Template - Design Mode", id="main")
        yield Footer()

In [ ]:
#| export
if __name__ == "__main__":
    app = TUITemplate()
    app.run()